In [ ]:
# Core Python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
session.sql("USE ROLE TRAINING_ROLE").collect() 
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

In [ ]:
records        = session.table("PERMIT_RECORDS").to_pandas()
census_tracts  = session.table("CENSUS_TRACTS").to_pandas()
contractors    = session.table("MASTER_LICENSE").to_pandas()
personnel      = session.table("PERSONNEL_DATA").to_pandas()
worker_comp    = session.table("WORKER_COMP").to_pandas()

In [ ]:
# 3.1 Dates
for col in ["ISSUE_DATE", "STATUS_DATE"]:
    records[col] = pd.to_datetime(records[col], errors="coerce")

# 3.2 Valuation (string -> numeric)
records["VALUATION"] = pd.to_numeric(
        records["VALUATION"].str.replace("$", "").str.replace(",",""),
        errors="coerce"
    )

# 3.3 How ot get a specific permit type, for example here we get HVAC (Heating, ventilation, and air conditioning)

hvac = records.loc[records["PERMIT_TYPE"].fillna("").str.upper() == "HVAC"]

# 3.5 Derive CT (LA‑specific tract key) from CENSUS_TRACT if present
ct_numeric = pd.to_numeric(records["CENSUS_TRACT"], errors="coerce")
records["CT"] = (ct_numeric * 100) + 6037000000

In [ ]:

records_with_census = records.merge(census_tracts, left_on="CT", right_on="CENSUS_TRACT", how="inner")

In [ ]:
hvac_with_contractors = pd.DataFrame()


hvac_by_contractor = (
            hvac.groupby("LICENSE_NUM")
                .agg(Projects=("PERMIT_SUB_TYPE", "count"),
                     TotalValue=("VALUATION", "sum"),
                     FirstYear=("ISSUE_DATE", "min"),
                     LastYear=("ISSUE_DATE", "max"))
                .reset_index()
        )

hvac_with_contractors = hvac_by_contractor.merge(
            contractors, left_on="LICENSE_NUM", right_on="LICENSE_NO", how="left"
        )


In [ ]:

pool_permits = records[records["AI_DESCRIPTION"].fillna("").str.contains("swimming pool", case=False)].copy()
pool_permits["year"] = pool_permits["ISSUE_DATE"].dt.year

counts_by_year = pool_permits.groupby("year")["AI_DESCRIPTION"].count().rename("num_permits")
value_by_year  = pool_permits.groupby("year")["VALUATION"].sum().rename("total_valuation") 




plt.figure()
counts_by_year.sort_index().plot(kind="bar")
plt.title("Swimming Pool Permits per Year")
plt.xlabel("Year")
plt.ylabel("Number of Permits")
plt.tight_layout()
plt.show()


In [ ]:
pool_by_zip = (pool_permits.groupby("ZIP_CODE")["AI_DESCRIPTION"]
                   .count()
                   .rename("num_pool_permits")
                   .sort_values(ascending=False)
                   .head(15))



plt.figure()
pool_by_zip.sort_values(ascending=True).plot(kind="barh")
plt.title("Top ZIP Codes by Pool Permits")
plt.xlabel("Number of Permits")
plt.ylabel("ZIP Code")
plt.tight_layout()
plt.show()


In [ ]:
group = "AMI_CATEGORY"
kitchen = records_with_census[
        records_with_census["AI_DESCRIPTION"].fillna("").str.contains("kitchen")
    ].copy()

kitchen_by_ami = kitchen[group].value_counts().rename("permits").sort_index()
hh_by_ami = (records_with_census.groupby(group)["NUM_HOUSEHOLDS"]
                     .sum()
                     .rename("households")
                     .sort_index())

dist = pd.concat([kitchen_by_ami / kitchen_by_ami.sum(),
                          hh_by_ami / hh_by_ami.sum()], axis=1)
dist.columns = ["permit_share", "household_share"]


# Compare via simple side-by-side bars (two panels)
ax = dist.plot(kind="bar")
ax.set_title("Kitchen Permits vs Household Share by AMI Category")
ax.set_xlabel("AMI Category")
ax.set_ylabel("Share")
plt.tight_layout()
plt.show()


In [ ]:


# Pool filter
pool = records[records["AI_DESCRIPTION"].fillna("").str.contains("swimming pool")].copy()

# Sidebar filters
st.sidebar.header("Filters")
min_dt = pd.to_datetime(pool["ISSUE_DATE"]).min().date()
max_dt = pd.to_datetime(pool["ISSUE_DATE"]).max().date()
date_range = st.sidebar.date_input("Issue date range", value=(min_dt, max_dt), min_value=min_dt, max_value=max_dt)


zip_options = sorted(pool["ZIP_CODE"].dropna().unique().tolist()) if "ZIP_CODE" in pool.columns else []
selected_zips = st.sidebar.multiselect("ZIP codes", options=zip_options, default=zip_options[:5] if zip_options else [])

# Apply filters
mask = pd.Series(True, index=pool.index)
if date_range:
    start, end = date_range
    mask &= pool["ISSUE_DATE"].dt.date.between(start, end)

if selected_zips:
    mask &= pool["ZIP_CODE"].isin(selected_zips)

filtered = pool.loc[mask].copy()

st.subheader("Filtered Pool Permits")
st.write(f"{len(filtered):,} permits match your filters.")

cols = ["PCIS_PERMIT_NUM","ISSUE_DATE","STATUS","VALUATION","AI_DESCRIPTION","ADDRESS_START","STREET_NAME","ZIP_CODE"] 
st.dataframe(filtered[cols].sort_values("ISSUE_DATE", ascending=False), use_container_width=True)


# Coordinates for map (LATITUDE_LONGITUDE like '(lon lat)')

s=filtered['LATITUDE_LONGITUDE'].dropna()
coords=s.str.extract(r'\(([-\d.]+) ([-\d.]+)\)').rename(columns={0:'longitude',1:'latitude'}).astype(float)

st.subheader("Map")
st.map(coords)

In [ ]:
# ------------------------------
# IMPORTS
# ------------------------------
import pandas as pd
import numpy as np
import streamlit as st
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# ------------------------------
# LOAD DATA
# ------------------------------
session.sql("USE ROLE TRAINING_ROLE").collect()
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

records        = session.table("PERMIT_RECORDS").to_pandas()
census_tracts  = session.table("CENSUS_TRACTS").to_pandas()
contractors    = session.table("MASTER_LICENSE").to_pandas()

# ------------------------------
# BASIC CLEANING
# ------------------------------
for col in ["ISSUE_DATE", "STATUS_DATE"]:
    records[col] = pd.to_datetime(records[col], errors="coerce")

records["VALUATION"] = pd.to_numeric(
    records["VALUATION"].astype(str)
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False),
    errors="coerce"
)

# CT join key
ct_numeric = pd.to_numeric(records["CENSUS_TRACT"], errors="coerce")
records["CT"] = (ct_numeric * 100) + 6037000000

records_with_census = records.merge(
    census_tracts,
    left_on="CT",
    right_on="CENSUS_TRACT",
    how="left"
)

# ------------------------------
# GLOBAL DASHBOARD TITLE
# ------------------------------
st.title(" LA Building Permit Insights Dashboard")

st.write(
    "Interactive multi-tab dashboard exploring LA building permits, "
    "including luxury improvements, income inequality, contractor trends, and more."
)

# ------------------------------
# SIDEBAR FILTERS (GLOBAL)
# ------------------------------
st.sidebar.header(" Global Filters")

# Date filter
min_date = records["ISSUE_DATE"].min().date()
max_date = records["ISSUE_DATE"].max().date()

date_range = st.sidebar.date_input(
    "Filter by Issue Date Range",
    (min_date, max_date),
    min_value=min_date,
    max_value=max_date
)

# ZIP filter
zip_options = sorted(records["ZIP_CODE"].dropna().unique().tolist())
selected_zips = st.sidebar.multiselect(
    "Filter by ZIP Codes",
    options=zip_options,
    default=[]
)

# Permit Type filter
permit_type_options = sorted(records["PERMIT_TYPE"].dropna().unique().tolist())
selected_permit_types = st.sidebar.multiselect(
    "Permit Types",
    options=permit_type_options,
    default=[]
)

# AMI Income Category filter
ami_options = sorted(records_with_census["AMI_CATEGORY"].dropna().unique().tolist())
selected_ami = st.sidebar.multiselect(
    "AMI Income Categories",
    options=ami_options,
    default=[]
)

# ------------------------------
# APPLY GLOBAL FILTERS
# ------------------------------
filtered = records.copy()

if date_range:
    start, end = date_range
    filtered = filtered[
        filtered["ISSUE_DATE"].dt.date.between(start, end)
    ]

if selected_zips:
    filtered = filtered[filtered["ZIP_CODE"].isin(selected_zips)]

if selected_permit_types:
    filtered = filtered[filtered["PERMIT_TYPE"].isin(selected_permit_types)]

if selected_ami:
    filtered = filtered.merge(
        census_tracts[["CENSUS_TRACT", "AMI_CATEGORY"]],
        left_on="CT",
        right_on="CENSUS_TRACT",
        how="left"
    )
    filtered = filtered[filtered["AMI_CATEGORY"].isin(selected_ami)]

st.write(f"### Showing {len(filtered):,} filtered permits")

# ------------------------------
# TABS
# ------------------------------
tab1, tab2, tab3, tab4, tab5 = st.tabs([
    " Pool Permits",
    " Kitchen Remodels",
    " Contractors",
    " Income Inequality",
    " All Permit Trends"
])

# ---------------------------------------------------------
# TAB 1 — POOL PERMITS
# ---------------------------------------------------------
with tab1:
    pool = filtered[
        filtered["AI_DESCRIPTION"].fillna("").str.contains("swimming pool", case=False)
    ].copy()

    st.subheader(" Pool Permits Overview")

    st.metric("Total Pool Permits", len(pool))

    if len(pool) > 0:
        pool["year"] = pool["ISSUE_DATE"].dt.year

        st.write("#### Pool Permits by Year")
        st.bar_chart(pool.groupby("year")["AI_DESCRIPTION"].count())

        # Top ZIPS
        st.write("#### Top ZIP Codes for Pool Permits")
        top_zips = pool.groupby("ZIP_CODE")["AI_DESCRIPTION"].count().sort_values(ascending=False).head(10)
        st.bar_chart(top_zips)

        # Map
        st.write("#### Pool Permit Locations")
        coords = pool["LATITUDE_LONGITUDE"].dropna().str.extract(
            r"\(([-\d.]+) ([-\d.]+)\)"
        ).astype(float)
        coords.columns = ["longitude", "latitude"]
        st.map(coords)

# ---------------------------------------------------------
# TAB 2 — KITCHEN REMODELS
# ---------------------------------------------------------
with tab2:
    kitchen = filtered[
        filtered["AI_DESCRIPTION"].fillna("").str.contains("kitchen", case=False)
    ].copy()

    st.subheader("🍳 Kitchen Renovations Overview")

    st.metric("Total Kitchen Permits", len(kitchen))

    if len(kitchen) > 0:
        kitchen["year"] = kitchen["ISSUE_DATE"].dt.year

        st.write("#### Kitchen Permits by Year")
        st.line_chart(kitchen.groupby("year")["AI_DESCRIPTION"].count())

        st.write("#### Average Valuation of Kitchen Permits by Year")
        st.line_chart(kitchen.groupby("year")["VALUATION"].mean())

# ---------------------------------------------------------
# TAB 3 — CONTRACTORS
# ---------------------------------------------------------
with tab3:
    st.subheader(" Contractor Activity")

    if "LICENSE_NUM" in filtered.columns:
        contractor_stats = (
            filtered.groupby("LICENSE_NUM")
            .agg(
                num_projects=("PERMIT_SUB_TYPE", "count"),
                total_value=("VALUATION", "sum"),
                first_year=("ISSUE_DATE", lambda x: x.min().year),
                last_year=("ISSUE_DATE", lambda x: x.max().year)
            )
            .sort_values("num_projects", ascending=False)
            .head(20)
        )
        st.dataframe(contractor_stats)

# ---------------------------------------------------------
# TAB 4 — INCOME INEQUALITY
# ---------------------------------------------------------
with tab4:
    st.subheader(" Income Group Comparison")

    merged = filtered.merge(
        census_tracts[["CENSUS_TRACT", "AMI_CATEGORY", "NUM_HOUSEHOLDS"]],
        left_on="CT",
        right_on="CENSUS_TRACT",
        how="left"
    )

    permits_by_ami = merged.groupby("AMI_CATEGORY")["PERMIT_TYPE"].count()
    households = merged.groupby("AMI_CATEGORY")["NUM_HOUSEHOLDS"].sum()

    share_df = pd.concat([
        permits_by_ami / permits_by_ami.sum(),
        households / households.sum()
    ], axis=1)
    share_df.columns = ["permit_share", "household_share"]

    st.write("### Permit Share vs Household Share by AMI (Income Category)")
    st.bar_chart(share_df)
    st.dataframe(share_df)

# ---------------------------------------------------------
# TAB 5 — ALL PERMIT TRENDS
# ---------------------------------------------------------
with tab5:
    st.subheader(" General Permit Trends")

    filtered["year"] = filtered["ISSUE_DATE"].dt.year

    st.write("#### Total Permits by Year")
    st.line_chart(filtered.groupby("year")["PERMIT_TYPE"].count())

    st.write("#### Total Valuation by Year")
    st.line_chart(filtered.groupby("year")["VALUATION"].sum())

    st.write("#### Permit Count by Permit Type")
    top_types = filtered["PERMIT_TYPE"].value_counts().head(15)
    st.bar_chart(top_types)


In [ ]:
# ------------------------------
# IMPORTS
# ------------------------------
import pandas as pd
import numpy as np
import streamlit as st
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# ------------------------------
# LOAD TABLES
# ------------------------------
session.sql("USE ROLE TRAINING_ROLE").collect()
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

records        = session.table("PERMIT_RECORDS").to_pandas()
census_tracts  = session.table("CENSUS_TRACTS").to_pandas()
contractors    = session.table("MASTER_LICENSE").to_pandas()

# ------------------------------
# CLEANING
# ------------------------------
for col in ["ISSUE_DATE", "STATUS_DATE"]:
    records[col] = pd.to_datetime(records[col], errors="coerce")

records["VALUATION"] = pd.to_numeric(
    records["VALUATION"].astype(str).str.replace("$","",regex=False).str.replace(",","",regex=False),
    errors="coerce"
)

ct_num = pd.to_numeric(records["CENSUS_TRACT"], errors="coerce")
records["CT"] = (ct_num * 100) + 6037000000

records_with_census = records.merge(
    census_tracts,
    left_on="CT",
    right_on="CENSUS_TRACT",
    how="left"
)

# ----------------------------------------------------------
# DASHBOARD UI
# ----------------------------------------------------------
st.title(" LA Building Permit Insights Dashboard")

# ----------------------------------------------------------
# GLOBAL FILTERS
# ----------------------------------------------------------
st.sidebar.header("GLOBAL FILTERS (Apply to All Tabs)")

min_date = records["ISSUE_DATE"].min().date()
max_date = records["ISSUE_DATE"].max().date()

global_date = st.sidebar.date_input(
    "Date Range", (min_date, max_date), min_value=min_date, max_value=max_date
)

global_zip = st.sidebar.multiselect(
    "ZIP Filter",
    sorted(records["ZIP_CODE"].dropna().unique().tolist()),
)

global_types = st.sidebar.multiselect(
    "Permit Types",
    sorted(records["PERMIT_TYPE"].dropna().unique().tolist()),
)

# Apply global filters
filtered_global = records.copy()

# date
if global_date:
    start, end = global_date
    filtered_global = filtered_global[filtered_global["ISSUE_DATE"].dt.date.between(start, end)]

# zip
if global_zip:
    filtered_global = filtered_global[filtered_global["ZIP_CODE"].isin(global_zip)]

# permit type
if global_types:
    filtered_global = filtered_global[filtered_global["PERMIT_TYPE"].isin(global_types)]

# ----------------------------------------------------------
# TABS
# ----------------------------------------------------------
tab1, tab2, tab3, tab4, tab5 = st.tabs([
    " Pools",
    " Kitchens",
    " Contractors",
    " Income Inequality",
    " All Permit Trends"
])

# ----------------------------------------------------------
# TAB 1 — POOLS
# ----------------------------------------------------------
with tab1:
    st.header(" Swimming Pool Permits")

    # Per-graph filters
    st.subheader("Filters for Pool Visualizations")
    
    pool_year = st.slider(
        "Select Year Range",
        int(filtered_global["ISSUE_DATE"].dt.year.min()),
        int(filtered_global["ISSUE_DATE"].dt.year.max()),
        (2010, 2024)
    )

    pool_zip = st.multiselect(
        "ZIP (Pool Tab Only)",
        sorted(filtered_global["ZIP_CODE"].dropna().unique().tolist())
    )

    keyword = st.text_input("Keyword filter in description", "swimming pool")

    # Apply local filters
    df = filtered_global.copy()
    df = df[df["AI_DESCRIPTION"].fillna("").str.contains(keyword, case=False)]
    df = df[df["ISSUE_DATE"].dt.year.between(pool_year[0], pool_year[1])]

    if pool_zip:
        df = df[df["ZIP_CODE"].isin(pool_zip)]

    st.metric("Total Pool Permits (After Filters)", len(df))

    # Graph 1: Pool Permits by Year
    df["year"] = df["ISSUE_DATE"].dt.year
    st.subheader("Pool Permits by Year")
    st.bar_chart(df.groupby("year")["PERMIT_TYPE"].count())

    # Graph 2: Top ZIPs
    st.subheader("Top ZIPs for Pool Permits")
    top_zip = df.groupby("ZIP_CODE")["PERMIT_TYPE"].count().sort_values(ascending=False).head(10)
    st.bar_chart(top_zip)

    # Graph 3: Map
    st.subheader("Pool Permit Locations")
    coords = df["LATITUDE_LONGITUDE"].dropna().str.extract(r"\(([-\d.]+) ([-\d.]+)\)").astype(float)
    coords.columns = ["longitude","latitude"]
    st.map(coords)

# ----------------------------------------------------------
# TAB 2 — KITCHEN REMODELS
# ----------------------------------------------------------
with tab2:
    st.header(" Kitchen Renovations")

    # Per-graph filters
    st.subheader("Filters for Kitchen Visualizations")

    kitchen_year = st.slider(
        "Kitchen Year Range",
        2010,
        2024,
        (2012, 2024)
    )

    min_val, max_val = int(records["VALUATION"].min()), int(records["VALUATION"].max())

    kitchen_val = st.slider(
        "Valuation Range",
        min_val, max_val,
        (min_val, max_val)
    )

    # Apply filters
    dfk = filtered_global.copy()
    dfk = dfk[dfk["AI_DESCRIPTION"].fillna("").str.contains("kitchen", case=False)]
    dfk = dfk[dfk["ISSUE_DATE"].dt.year.between(kitchen_year[0], kitchen_year[1])]
    dfk = dfk[dfk["VALUATION"].between(kitchen_val[0], kitchen_val[1])]

    st.metric("Total Kitchen Permits", len(dfk))

    # Graph 1
    dfk["year"] = dfk["ISSUE_DATE"].dt.year
    st.subheader("Kitchen Permits Over Time")
    st.line_chart(dfk.groupby("year")["PERMIT_TYPE"].count())

    # Graph 2
    st.subheader("Average Kitchen Valuation")
    st.line_chart(dfk.groupby("year")["VALUATION"].mean())

# ----------------------------------------------------------
# TAB 3 — CONTRACTORS
# ----------------------------------------------------------
with tab3:
    st.header(" Contractor Performance Dashboard")

    # NEW FILTER: Choose contractor
    contractor_list = sorted(contractors["LICENSE_NO"].dropna().unique().tolist())

    selected_cons = st.multiselect(
        "Select Contractors",
        contractor_list
    )

    dfc = filtered_global.copy()

    if selected_cons:
        dfc = dfc[dfc["LICENSE_NUM"].isin(selected_cons)]

    st.metric("Permits by Selected Contractors", len(dfc))

    contractor_stats = (
        dfc.groupby("LICENSE_NUM")
        .agg(
            num_projects=("PERMIT_SUB_TYPE","count"),
            total_valuation=("VALUATION","sum"),
            first_year=("ISSUE_DATE",lambda x:x.min().year),
            last_year=("ISSUE_DATE",lambda x:x.max().year)
        )
    )

    st.dataframe(contractor_stats)

# ----------------------------------------------------------
# TAB 4 — INCOME INEQUALITY
# ----------------------------------------------------------
with tab4:
    st.header(" Permits by Income Group (AMI)")

    # Local AMI filter
    ami_local = st.multiselect(
        "AMI Categories (Tab Filter)",
        sorted(records_with_census["AMI_CATEGORY"].dropna().unique().tolist())
    )

    dfi = filtered_global.merge(
        census_tracts[["CENSUS_TRACT","AMI_CATEGORY","NUM_HOUSEHOLDS"]],
        left_on="CT",
        right_on="CENSUS_TRACT",
        how="left"
    )

    if ami_local:
        dfi = dfi[dfi["AMI_CATEGORY"].isin(ami_local)]

    permits_by_ami = dfi.groupby("AMI_CATEGORY")["PERMIT_TYPE"].count()
    households = dfi.groupby("AMI_CATEGORY")["NUM_HOUSEHOLDS"].sum()

    ratio = pd.concat([
        permits_by_ami / permits_by_ami.sum(),
        households / households.sum()
    ], axis=1)
    ratio.columns = ["permit_share","household_share"]

    st.bar_chart(ratio)
    st.dataframe(ratio)

# ----------------------------------------------------------
# TAB 5 — ALL PERMIT TRENDS
# ----------------------------------------------------------
with tab5:
    st.header(" Overall Trends")

    # Local filter: choose permit type
    permit_local = st.multiselect(
        "Filter Permit Types Inside This Tab",
        sorted(records["PERMIT_TYPE"].dropna().unique().tolist())
    )

    dfa = filtered_global.copy()
    if permit_local:
        dfa = dfa[dfa["PERMIT_TYPE"].isin(permit_local)]

    dfa["year"] = dfa["ISSUE_DATE"].dt.year

    st.subheader("Total Permits by Year")
    st.line_chart(dfa.groupby("year")["PERMIT_TYPE"].count())

    st.subheader("Total Valuation by Year")
    st.line_chart(dfa.groupby("year")["VALUATION"].sum())
